## Assignment Questions
1. What are the distinct number of meal categories andcuisines?
2. Which center_id has the highest num_orders?
3. What is the top selling cuisine at the center_id thathad the highest num_orders?
4. What is the average op_area per center_type?
5. Which center_type had the highest revenue? (Revenueis total sum of
checkout_price*num_orders)
6. Which is the top ordered cuisine in terms of num_orders?
7. What are the num_orders per cuisine per week?
8. Which center_id gave the highest number of discounts?(Discount is considered
when checkout_price is less than base_price)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()
spark

22/07/03 17:47:09 WARN Utils: Your hostname, codespaces-ae9593 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/07/03 17:47:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/03 17:47:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/03 17:47:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
demand_data = spark.read.csv('data/train.csv',inferSchema=True,header=True) # Creating dataframe from the data

demand_data.createOrReplaceTempView("demand")

In [3]:
demand_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- center_id: integer (nullable = true)
 |-- meal_id: integer (nullable = true)
 |-- checkout_price: double (nullable = true)
 |-- base_price: double (nullable = true)
 |-- emailer_for_promotion: integer (nullable = true)
 |-- homepage_featured: integer (nullable = true)
 |-- num_orders: integer (nullable = true)



In [4]:
spark.sql("select * from demand").show() 

+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|     id|week|center_id|meal_id|checkout_price|base_price|emailer_for_promotion|homepage_featured|num_orders|
+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|1379560|   1|       55|   1885|        136.83|    152.29|                    0|                0|       177|
|1466964|   1|       55|   1993|        136.83|    135.83|                    0|                0|       270|
|1346989|   1|       55|   2539|        134.86|    135.86|                    0|                0|       189|
|1338232|   1|       55|   2139|         339.5|    437.53|                    0|                0|        54|
|1448490|   1|       55|   2631|         243.5|     242.5|                    0|                0|        40|
|1270037|   1|       55|   1248|        251.23|    252.23|                    0|                0|        28|
|1191377| 

In [5]:
fullfillment_centre = spark.read.csv('data/fulfilment_center_info.csv',inferSchema=True,header=True) # Creating dataframe from the data

fullfillment_centre.createOrReplaceTempView("centre")

In [6]:
fullfillment_centre.printSchema()

root
 |-- center_id: integer (nullable = true)
 |-- city_code: integer (nullable = true)
 |-- region_code: integer (nullable = true)
 |-- center_type: string (nullable = true)
 |-- op_area: double (nullable = true)



In [7]:
spark.sql("select * from centre").show() 

+---------+---------+-----------+-----------+-------+
|center_id|city_code|region_code|center_type|op_area|
+---------+---------+-----------+-----------+-------+
|       11|      679|         56|     TYPE_A|    3.7|
|       13|      590|         56|     TYPE_B|    6.7|
|      124|      590|         56|     TYPE_C|    4.0|
|       66|      648|         34|     TYPE_A|    4.1|
|       94|      632|         34|     TYPE_C|    3.6|
|       64|      553|         77|     TYPE_A|    4.4|
|      129|      593|         77|     TYPE_A|    3.9|
|      139|      693|         34|     TYPE_C|    2.8|
|       88|      526|         34|     TYPE_A|    4.1|
|      143|      562|         77|     TYPE_B|    3.8|
|      101|      699|         85|     TYPE_C|    2.8|
|       86|      699|         85|     TYPE_C|    4.0|
|       32|      526|         34|     TYPE_A|    3.8|
|      149|      478|         77|     TYPE_A|    2.4|
|      152|      576|         34|     TYPE_B|    4.0|
|       92|      526|       

In [8]:
meal_info = spark.read.csv('data/meal_info.csv',inferSchema=True,header=True) # Creating dataframe from the data

meal_info.createOrReplaceTempView("meal")

In [9]:
meal_info.printSchema()

root
 |-- meal_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- cuisine: string (nullable = true)



In [10]:
spark.sql("select * from meal").show()

+-------+------------+-------+
|meal_id|    category|cuisine|
+-------+------------+-------+
|   1885|   Beverages|   Thai|
|   1993|   Beverages|   Thai|
|   2539|   Beverages|   Thai|
|   1248|   Beverages| Indian|
|   2631|   Beverages| Indian|
|   1311|      Extras|   Thai|
|   1062|   Beverages|Italian|
|   1778|   Beverages|Italian|
|   1803|      Extras|   Thai|
|   1198|      Extras|   Thai|
|   2707|   Beverages|Italian|
|   1847|        Soup|   Thai|
|   1438|        Soup|   Thai|
|   2494|        Soup|   Thai|
|   2760|Other Snacks|   Thai|
|   2490|       Salad|Italian|
|   1109|   Rice Bowl| Indian|
|   2290|   Rice Bowl| Indian|
|   1525|Other Snacks|   Thai|
|   2704|Other Snacks|   Thai|
+-------+------------+-------+
only showing top 20 rows



### 1. What are the distinct number of meal categories and cuisines?


In [11]:
## 1. What are the distinct number of meal categories
print("The distinct no of meal categories are: ",spark.sql("select distinct(category) from meal").count())
spark.sql("select distinct(category) from meal").show()

The distinct no of meal categories are:  14
+------------+
|    category|
+------------+
|       Salad|
|      Desert|
|     Biryani|
|   Rice Bowl|
|    Sandwich|
|       Pizza|
|   Beverages|
|Other Snacks|
|        Soup|
|    Starters|
|      Extras|
|     Seafood|
|       Pasta|
|        Fish|
+------------+



In [12]:
## 1. What are the distinct number of cuisines 
print("The distinct no of meal cusines are: ", spark.sql("select distinct(cuisine) from meal").count())
spark.sql("select distinct(cuisine) from meal").show()

The distinct no of meal cusines are:  4
+-----------+
|    cuisine|
+-----------+
|       Thai|
|     Indian|
|Continental|
|    Italian|
+-----------+



### 2. Which center_id has the highest num_orders?

In [13]:
k=(demand_data.groupBy("center_id").agg(F.sum("num_orders").alias("Total_Orders"))).orderBy("Total_Orders", ascending=False)
j = ((k.collect())[0]).asDict()
print(f"The Centre with the highest orders is {j['center_id']}, with total orders = {j['Total_Orders']}")
k.show(1)

The Centre with the highest orders is 13, with total orders = 1742220
+---------+------------+
|center_id|Total_Orders|
+---------+------------+
|       13|     1742220|
+---------+------------+
only showing top 1 row



In [14]:
# Alternative way
statmt = '''
SELECT center_id,SUM(num_orders) AS Total_Orders
FROM demand
GROUP BY center_id
ORDER BY SUM(num_orders) DESC
LIMIT 1
'''
k = spark.sql(statmt).show()

+---------+------------+
|center_id|Total_Orders|
+---------+------------+
|       13|     1742220|
+---------+------------+



### 3. What is the top selling cuisine at the center_id that had the highest num_orders?

In [41]:
statmt = '''
SELECT center_id,num_orders,cuisine 
FROM demand,meal 
WHERE demand.meal_id = meal.meal_id 
ORDER BY num_orders DESC
'''
k = spark.sql(statmt)
j = ((k.collect())[0]).asDict()
print(f"The Top selling Cuisine being {j['cuisine']} , the Centre with the highest orders is {j['center_id']}, with total orders = {j['num_orders']}")
k.show()

The Top selling Cuisine being Indian , the Centre with the highest orders is 43, with total orders = 24299
+---------+----------+-------+
|center_id|num_orders|cuisine|
+---------+----------+-------+
|       43|     24299| Indian|
|       43|     15336| Indian|
|       43|     14229| Indian|
|       10|     13580| Indian|
|       89|     12489| Indian|
|      146|     12327|Italian|
|      137|     12177| Indian|
|       99|     12137| Indian|
|      126|     11380|Italian|
|       11|     11260| Indian|
|      104|     11246| Indian|
|       13|     10745| Indian|
|       51|     10449| Indian|
|       52|     10435| Indian|
|      108|     10259| Indian|
|       99|     10193| Indian|
|      113|     10124| Indian|
|       59|      9828| Indian|
|       99|      9815| Indian|
|      113|      9532|Italian|
+---------+----------+-------+
only showing top 20 rows



### 4. What is the average op_area per center_type?

In [16]:
statmt = '''
SELECT center_type, AVG(op_area) AS AVG_OP_AREA
FROM centre
GROUP BY center_type
'''
k = spark.sql(statmt).show()

+-----------+------------------+
|center_type|       AVG_OP_AREA|
+-----------+------------------+
|     TYPE_C|3.1578947368421044|
|     TYPE_B|4.7733333333333325|
|     TYPE_A| 4.076744186046512|
+-----------+------------------+



### 5. Which center_type had the highest revenue? (Revenue is total sum of checkout_price*num_orders)

In [22]:
statmt = '''
SELECT c.center_type,SUM(d.checkout_price * d.num_orders) AS Revenue
FROM centre c
JOIN demand d ON c.center_id = d.center_id
GROUP BY c.center_type
ORDER BY Revenue DESC
LIMIT 1
'''
k = spark.sql(statmt)
j = ((k.collect())[0]).asDict()
print(f"The Center Type which has the highest revenue is {j['center_type']}, the revenue being {j['Revenue']}")
k.show()

The Center Type which has the highest revenue is TYPE_A, the revenue being 7276203201.870064
+-----------+-------------------+
|center_type|            Revenue|
+-----------+-------------------+
|     TYPE_A|7.276203201870064E9|
+-----------+-------------------+



In [20]:
# Alternate way
statmt = '''
SELECT center_type, SUM(checkout_price*num_orders) AS Revenue
FROM centre,demand
WHERE centre.center_id = demand.center_id
GROUP BY center_type
ORDER BY Revenue DESC
'''
k = spark.sql(statmt).show()

+-----------+-------------------+
|center_type|            Revenue|
+-----------+-------------------+
|     TYPE_A|7.276203201870064E9|
|     TYPE_B|3.172968529400033E9|
|     TYPE_C|2.251833991370006E9|
+-----------+-------------------+



### 6. Which is the top ordered cuisine in terms of num_orders?

In [32]:
statmt = '''
SELECT m.cuisine, SUM(d.num_orders) AS Total_Orders
FROM meal m
JOIN demand d ON m.meal_id = d.meal_id
GROUP BY m.cuisine
ORDER BY SUM(d.num_orders) DESC
'''
k = spark.sql(statmt)
j = ((k.collect())[0]).asDict()
print(f"The Tope Ordered Cuisine is {j['cuisine']}, the total No of Orders being {j['Total_Orders']}")
k.show()

The Tope Ordered Cuisine is Italian, the total No of Orders being 17166334
+-----------+------------+
|    cuisine|Total_Orders|
+-----------+------------+
|    Italian|    17166334|
|       Thai|    14058488|
|     Indian|    10979934|
|Continental|     6766188|
+-----------+------------+



### 7. What are the num_orders per cuisine per week?

In [34]:
statmt = '''
SELECT d.week, m.cuisine, SUM(d.num_orders) AS Total_Orders
FROM meal m
JOIN demand d ON m.meal_id = d.meal_id
GROUP BY d.week,m.cuisine
ORDER BY d.week
'''
k = spark.sql(statmt).show()

+----+-----------+------------+
|week|    cuisine|Total_Orders|
+----+-----------+------------+
|   1|Continental|      146020|
|   1|     Indian|      175317|
|   1|    Italian|      228836|
|   1|       Thai|      242088|
|   2|     Indian|      177109|
|   2|       Thai|      273778|
|   2|Continental|      133570|
|   2|    Italian|      202627|
|   3|    Italian|      197299|
|   3|     Indian|      150148|
|   3|       Thai|      249838|
|   3|Continental|       97977|
|   4|    Italian|      192265|
|   4|       Thai|      277206|
|   4|     Indian|      155239|
|   4|Continental|      118819|
|   5|Continental|      116077|
|   5|    Italian|      169161|
|   5|     Indian|      683532|
|   5|       Thai|      229905|
+----+-----------+------------+
only showing top 20 rows



### 8. Which center_id gave the highest number of discounts?(Discount is considered when checkout_price is less than base_price)

In [40]:
statmt = '''
SELECT center_id, SUM(base_price - checkout_price) AS Total_Discount, COUNT(*) AS No_Of_Discounts
FROM demand
WHERE base_price > checkout_price
GROUP BY center_id
ORDER BY COUNT(*) DESC
'''
k = spark.sql(statmt)
j = ((k.collect())[0]).asDict()
print(f"The Center Id for which there was highest no of Discounts is {j['center_id']}, they gave discounts a total of {j['No_Of_Discounts']} times, the Total Discount they gave being {j['Total_Discount']}.")
k.show()

The Center Id for which there was highest no of Discounts is 13, they gave discounts a total of 1509 times, the Total Discount they gave being 61917.64000000004.
+---------+------------------+---------------+
|center_id|    Total_Discount|No_Of_Discounts|
+---------+------------------+---------------+
|       13| 61917.64000000004|           1509|
|       30| 71262.66000000002|           1495|
|      137| 60545.02000000001|           1462|
|       27| 63096.98000000007|           1462|
|      153| 60795.27000000003|           1455|
|       10|58329.800000000025|           1454|
|       51|63354.630000000034|           1453|
|      174|64248.720000000016|           1447|
|       36|58773.280000000006|           1445|
|      104|          59457.86|           1443|
|       11| 60545.38999999998|           1439|
|      132|62439.079999999994|           1435|
|       43|60019.820000000014|           1431|
|      108| 61210.79000000001|           1429|
|       89| 62003.43000000002|         